In [2]:
import pydicom
import tkinter as tk
from tkinter import *
import numpy as np
from tkinter import filedialog as tkFileDialog
from tkinter.filedialog import askdirectory
import cv2 
import PIL
from PIL import ImageTk,Image, ImageDraw, ImageFont
import os
import fnmatch
import datetime
import matplotlib.pyplot as plt
import speech_recognition as sr 
import pyttsx3 
import textwrap

class MyWindow(Frame):
    def __init__(self, *args, **kwargs):
        Frame.__init__(self, *args, **kwargs)
        self.canvas = Canvas(self,width=450, height=500, bg='white')   
        self.canvas.place(x=0,y=60) 
        self.canvas.create_rectangle(10, 10, 440, 500, width=2)
        widget = Label(self.canvas, text='AMITY', fg='blue',font=('Times New Roman',38), anchor=CENTER)
        widget.place(x=168,y=15)
        widget = Label(self.canvas, text='UNIVERSITY', fg='blue',font=('Times New Roman',21), anchor=CENTER)
        widget.place(x=166, y=55)
        widget = Label(self.canvas, text='GURGAON', fg='blue',font=('Times New Roman',12), anchor=CENTER)
        widget.place(x=200,y=83)
        self.canvas.create_line(180, 93, 285, 93, fill='blue')
        widget = Label(self.canvas, text='LIVER SEGMENTATION GUI', fg='green', font=('Comic Sans MS',20), anchor=CENTER)
        widget.place(x=80,y=140)
        self.canvas.create_line(10, 230, 440, 230)
        widget = Label(self.canvas, text='Developed By :', font=('Arial',16), anchor=CENTER)
        widget.place(x=180,y=240)
        widget = Label(self.canvas, text='Ms. SAUMYA KANSAL', font=('Arial',16), anchor=CENTER)
        widget.place(x=150,y=260)
        widget = Label(self.canvas, text='    For :', font=('Arial',16), anchor=CENTER)
        widget.place(x=190,y=290)
        widget = Label(self.canvas, text='         MAJOR PROJECT', font=('Arial',16), anchor=CENTER)
        widget.place(x=120,y=310)
        self.canvas.create_line(10, 350, 440, 350)
        widget = Label(self.canvas, text='DEPARTMENT OF BIOMEDICAL ENGINEERING', font=('Times New Roman',15,'bold'), anchor=CENTER, fg='blue')
        widget.place(x=39,y=450)
        widget = Label(self.canvas, text='AMITY SCHOOL OF ENGINEERING AND TECHNOLOGY', font=('Times New Roman',15,'bold'), anchor=CENTER,fg='blue')
        widget.place(x=11,y=469)
        self.but=Button(self,text='select folder', command=self.enter)
        self.canvas.pack()
        self.but.pack()
    def save_jpg(self):
        filename = tkFileDialog.asksaveasfile(mode='w', defaultextension=".jpg")
        if not filename:
             return
        self.to_save=self.savi.convert('RGB')
        self.to_save.save(filename)
        
    def save_dicom(self):
        photo=self.save_d.copy()
        w,h=photo.size
        photo=np.asarray(photo)
        self.im.PixelData=photo.tostring()
        filename = tkFileDialog.asksaveasfile(mode='wb', defaultextension=".dcm")
        if not filename:
             return
        self.im.Rows=w
        self.im.Columns=h
        dt = datetime.datetime.now()
        self.im.ContentDate = dt.strftime('%Y%m%d')
        timeStr = dt.strftime('%H%M%S.%f')  # long format with micro seconds
        self.im.ContentTime = timeStr
        self.im.HighBit = 7
        self.im.BitsStored = 8
        self.im.BitsAllocated = 8
        self.im.save_as(filename)
    def enter(self):
        try:
            self.text_NO.pack_forget()
        except:
            True
        try:
            self.a.pack_forget()
        except:
            True
        try:
            self.b.pack_forget()
        except:
            True
        try:
            self.c.pack_forget()
        except:
            True
            
        self.but.pack_forget()
        self.canvas.destroy()
        def display():
            #self.title("slide"+str(sp.get()))
            a=int(sp.get())
            image=self.data[a]
            for n,x in enumerate(image):
                if x.any()!=0:
                    desired2=n
                    break
            if desired2==0:
                req_y=0
            else:
                req_y=desired2-100
            wrap=np.roll(image,-req_y, axis=0)
            self.imagetowork=wrap
            ixi=Image.fromarray(wrap)
            self.image_new=ixi
            render=ImageTk.PhotoImage(ixi)
            self.l.configure(image=render)
            self.l.image=render
            self.l.pack()
            self.a.configure(state='normal')
            self.c.configure(state='normal')
            
        def load_scan(path):
            im=[pydicom.dcmread(path+'/'+s) for s in os.listdir(path)]
            im = [s for s in im if 'SliceLocation' in s]
            im.sort(key = lambda x: int(x.InstanceNumber))
            try:
                slice_thickness = np.abs(im[0].ImagePositionPatient[2]-im[1].ImagePositionPatient[2])
            except:
                slice_thickness = np.abs(im[0].SliceLocation-im[1].SliceLocation)
            n=1
            self.slices=len(im)-1
            for s in im:
                if n==1:
                    self.checkmachine=s.Manufacturer
                    self.checkseries=s.SeriesDescription
                    self.slicethick=s.SliceThickness
                    n=0
            return im
        
        def get_pixels_hu(scans):
            image = np.stack([s.pixel_array for s in scans])
            image = image.astype(np.int16)
            #print(image)
            # Set outside-of-scan pixels to 0
            # The intercept is usually -1024, so air is approximately 0
            image[image == -2000] = 0 
            # Convert to Hounsfield units (HU)
            intercept = scans[0].RescaleIntercept
            slope = scans[0].RescaleSlope
            if slope != 1:
                image = slope * image.astype(np.float64)
                image = image.astype(np.int16)   
            image += np.int16(intercept)
            return np.array(image, dtype=np.int16)
        ifile = tkFileDialog.askdirectory(parent=self,title='Select folder')
        path = ifile
        scans=load_scan(path)
        if (self.checkmachine=="GE MEDICAL SYSTEMS" or self.checkseries=="AXIAL"):
            self.data=get_pixels_hu(scans)
            self.data[self.data<50]=0
            self.data[self.data>170]=0
            text=Label(self,wraplength=450,text='enter the slice number from 0 to {0}'.format(self.slices))
            text.pack()
            sp = Spinbox(self, from_= 0, to = 200)
            sp.pack()
            b3=Button(self, text='enter', command=display).pack()
            
        else:
            self.text_NO=Label(self,wraplength=450,text='ONLY VALID FOR GE MEDICAL SYSTEM and AXIAL SERIESDESCRIPTION')
            self.text_NO.pack()
            self.but.pack()
            
        self.a=Button(self,text='segmented liver and area', command=self.mor,state='disabled')
        self.a.pack()
        self.c=Button(self,text='livewire', command=self.livedraw,state='disabled')
        self.c.pack()
        self.l = Label(self)
        self.l.pack()
    def mor(self):
        def exit():
            self.mor.destroy()
        self.mor=Toplevel(self)
        x=self.imagetowork.copy()
        img=np.array(x, dtype=np.int16)
        kernel = np.ones((2,2),np.int16)
        Zin = cv2.morphologyEx(img , cv2.MORPH_OPEN, kernel)
        ret, thresh= cv2.threshold(Zin, 0, 200,cv2.THRESH_BINARY)
        mask = thresh.astype(np.uint8)
        ret,markers, stats, centeroid= cv2.connectedComponentsWithStats(mask,8)
        for x,c in enumerate(centeroid):
            if stats[x][4]<1100 or x==0 or x>180:
                thresh[markers==x]=0
                continue
            if abs(c[0]-c[1])<10:
                thresh[markers==x]=0
                continue
            if (c[0]<190 and c[0]>70 and c[1]<310 and c[1]>180):
                continue
            else:
                thresh[markers==x]=0
        
        #Hole filled liver mask
        imgi = thresh.astype('uint8')
        th, im_th = cv2.threshold(imgi, 0, 255, cv2.THRESH_BINARY);
        im_floodfill = im_th.copy()
        h, w = im_th.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(im_floodfill, mask, (0,0), 255)
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        thresh = im_th | im_floodfill_inv
        
        fin=self.imagetowork.copy()
        fin[thresh==False]=0
        
        lastly=np.asarray(fin)
        #print(lastly.shape)
        k = np.ones((2,2),np.uint8)
        gradient = cv2.morphologyEx(cv2.convertScaleAbs(lastly), cv2.MORPH_GRADIENT, k)        
        cont= cv2.findContours(gradient, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)[1]
        wrap=self.imagetowork.copy()
        cv2.drawContours(wrap, cont, -1,(255,255,255), thickness = 1)
        contour= max(cont, key=cv2.contourArea)
        area=cv2.contourArea(contour)*self.slicethick*0.001
        cv2.putText(wrap,'Area:'+str(area)+'cm2',(10,50),
                     cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),1)
        
        ixi=Image.fromarray(wrap)
        self.save_d=ixi
        array_im=[]
        array_im.append((wrap))
        self.to_save=array_im
        render=ImageTk.PhotoImage(ixi)
        mor=Label(self.mor,image=render)
        mor.image=render
        mor.pack()
        b=Button(self.mor, text='exit', command=exit).pack()
        def reti():
            self.notes()
        b1=Button(self.mor,text='NOTES',command=reti).pack()
        b2=Button(self.mor,text='save',command=self.save_dicom).pack()
        
       
    def notes(self):
        def docnotes(event):
            kiki=Toplevel(self)
            r=str(event.widget)
            if r[-1]=='l':
                c=0
            else:
                c=int(r[-1])-1
            xix=Image.fromarray(imi[c])
            renr=ImageTk.PhotoImage(xix)
            lon=Label(kiki,image=renr)
            lon.image=renr
            lon.grid(row=0, column=0,sticky = 'nw', pady = 2)
            def retrieve_input():
                def add_margin(pil_img, top, right, bottom, left, color):
                    width, height = pil_img.size
                    new_width = width + right + left
                    new_height = height + top + bottom
                    result = Image.new("RGB", (new_width, new_height), color)
                    result.paste(pil_img, (left, top))
                    return result
                im_new = add_margin(xix, 0, 500, 0, 0, (224, 255, 255))
                self.speek.configure(text="speak", state=DISABLED)
                inputValue=self.note.get("1.0","end-1c")
                self.note.grid_forget()
                lines = textwrap.wrap(inputValue, width=70)
                d1 = ImageDraw.Draw(im_new)
                y_h=30
                for line in lines:
                    line_height = 15
                    d1.text((540, y_h), line, fill=(0, 0, 0))
                    y_h+=line_height
                renrd=ImageTk.PhotoImage(im_new)
                self.savi=im_new
                lon.configure(image=renrd)
                lon.image=renrd
                butt.configure(text="edit",command=update)
                butt.grid(row=1, column=1,sticky = 'nw', pady = 2)
                self.butti=Button(kiki, height=1, width=25, text="SAVE", command=self.save_jpg)
                self.butti.grid(row=1, column=0,sticky = 'ne', pady = 2)
            def update():
                lon.configure(image=renr)
                lon.image=renr
                self.speek.configure(text="speak", state=NORMAL)
                self.butti.grid_forget()
                self.note.configure(bg = "light yellow")
                self.note.grid(row=0, column=1,sticky = 'nw', pady = 2)
                butt.configure(text="done",command=lambda: retrieve_input())
            self.note=Text(kiki, height=20, width=80,bg = "light yellow")
            butt=Button(kiki, height=1, width=25, text="Done",command=lambda: retrieve_input())
            self.note.grid(row=0, column=1,sticky = 'nw', pady = 2)
            butt.grid(row=1, column=1,sticky = 'nw', pady = 2)
            def spe(event):
                #print(event.type)
                r = sr.Recognizer()  
                T=True
                while(T):
                    self.speek.configure(text="listening...")
                    try:
                        with sr.Microphone() as source2:
                            r.adjust_for_ambient_noise(source2, duration=0.2)
                            audio2 = r.listen(source2)
                            MyText = r.recognize_google(audio2)
                            MyText = MyText.lower()
                            self.note.insert(END,MyText)
                    except sr.RequestError as e: 
                        self.note.insert(END,"Not connected to Internet\nCould not request results; {0}".format(e)) 
                    except sr.UnknownValueError: 
                        self.note.insert(END,"unknown error occured")
                    T=False
                self.speek.configure(text="speak")
            self.speek=Button(kiki, height=1, width=25, text='Speak')
            self.speek.bind('<Button-1>', spe)
            self.speek.grid(row=2, column=1,sticky = 'nw', pady = 2)
        l=[]
        koi=Toplevel(self)
        imi=self.to_save
        mic=0
        #print(len(imi))
        for i in range(len(imi)):
                mi=imi[i].copy()
                mi[mi!=1]=0
                mic+=mi
                xix=Image.fromarray(imi[i])
                render=ImageTk.PhotoImage(xix)
                l.append(Label(koi,image=render))
                l[i].image=render
                l[i].bind('<Button-1>',docnotes)
                l[i].grid(row=0, column=i,sticky = 'nw', pady = 2)
        a=np.asarray(self.image_new)
        mic = mic.astype('uint8')
        _,thresh=cv2.threshold(mic, 0, 2,cv2.THRESH_BINARY)
        co = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)[1]
        #xi=np.multiply(a,mic)
        a = a.astype('uint8')
        cv2.drawContours(a, co, -1,(2,2,2), thickness = 2)
        io=cv2.cvtColor(a,cv2.COLOR_GRAY2BGR)
        a=cv2.cvtColor(io, cv2.COLOR_BGR2GRAY)
        xi=Image.fromarray(a)
        self.save_d=xi
        ren=ImageTk.PhotoImage(xi)
        l=Label(koi,image=ren, text='Combined')
        l.image=ren
        l.grid(row=1, column=0,sticky = 'nw', pady = 2)
        butto=Button(koi,text='Save.dcm', command=self.save_dicom)
        butto.grid(row=1, column=1,sticky = 'e', pady = 2)
    
    def livedraw(self):
        def motion(event):
                event.widget.focus_set()
                self.points.append((event.x,event.y))
                if len(self.points)>1:
                    for i in range(0,len(self.points)-1):
                        cv2.line(self.live_image,(self.points[i][0],self.points[i][1]),(self.points[i+1][0],self.points[i+1][1]),(255,192,203),1)
                        ixi=Image.fromarray(self.live_image)
                        render=ImageTk.PhotoImage(ixi)
                        l1.configure(image=render)
                        l1.image=render
                
        def flag(event):
                cv2.line(self.live_image,(self.points[0][0],self.points[0][1]),(self.points[-1][0],self.points[-1][1]),(255,192,203),1)
                self.points=[]
                liv=self.live_image.copy()
                liv = liv.astype('uint8')
                liv[liv>1]=0
                liv[liv<1]=0
                self.live_image=np.asarray(self.image_new)
                _,thresh=cv2.threshold(liv, 0, 200,cv2.THRESH_BINARY)
                contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)[1]
                cv2.drawContours(self.nene, contours, -1,(255,192,203), thickness = 1 )
                xix=Image.fromarray(self.nene)
                render=ImageTk.PhotoImage(xix)
                l1.configure(image=render)
                l1.image=render
                b.configure(state='normal')
        def reti():
            self.to_save=array_im
            self.notes()
        def gogo(event):
            if event.char=='d':
                array_im.append((self.nene))
                self.nene=np.asarray(self.image_new)
        self.live=Toplevel(self)
        self.live.title('live wire')
        Text1=Label(self.live,wraplength=450,text='Press right button on mouse to join the points when done')
        Text2=Label(self.live,wraplength=450,text='Press (d) to save and mark another area')
        Text3=Label(self.live,wraplength=450,text='Click (SHOW) button to make notes for respective markings')
        Text1.pack()
        Text2.pack()
        Text3.pack()
        render=ImageTk.PhotoImage(self.image_new)
        l1=Label(self.live,image=render)
        l1.image=render
        self.live_image=np.asarray(self.image_new)
        self.nene=np.asarray(self.image_new)
        self.points=[]
        array_im=[]
        self.flag=True
        l1.bind('<Button-1>',motion)
        l1.bind('<Button-2>',flag)
        l1.bind('<Key>',lambda a: gogo(a))
        l1.pack()
        b=Button(self.live,text='SHOW',command=reti,state='disabled')
        b.pack()

window=tk.Tk()
mywin=MyWindow(window)
mywin.pack(side="top", fill="both", expand=True)
window.title('Dicom')
window.geometry("450x650+10+10")
window.mainloop()

ModuleNotFoundError: No module named 'cv2'